In [1]:
%load_ext autoreload
%autoreload
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import datetime
import time   
import math

from scipy.stats import binom, norm, t
from scipy.stats import ttest_ind_from_stats as ttest
from scipy.stats import chi2_contingency as chi2

from statsmodels.stats.power import NormalIndPower, tt_ind_solve_power
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

p_thresh = 0.1
default_typ2 = 0.2

from validate_calculate import *
from validate_stats import *
from validate_format import *


In [2]:
## The Split Python SDK

In [3]:
from splitio import get_factory

In [4]:
api_key = '9ea0fre07gg0m9eneoh31r6romoi0crrs2g8'

In [5]:

config = {'ready' : 5000,
         'eventsQueueSize' : 50000}

factory = get_factory(api_key, config=config)
split = factory.client()


## Mimic an AA test

## Randomizing user behaviour

In [6]:
booking_event_type = 'booking'
session_event = 'new_session'
traffic_type = 'user'
split_name = "SplitValidate" 
version = '2'

attributes = dict()
attributes['country'] = 'uk'

In [7]:
def getSessions(binom_n = 50, binom_p = 0.01, verbose=False):
    ns = 1 + binom.rvs(binom_n, binom_p)
    if verbose:
        print (str(datetime.datetime.now()), ns)
    return ns

def getBookings(booking_n = 100, booking_p = 0.005, verbose=False):
    nb = binom.rvs(booking_n, booking_p)
    if verbose: 
        print (str(datetime.datetime.now()), nb)
    return nb
def getBookingValue(booking_value_mean = 100, booking_value_standard_deviation = 15, verbose=False):
    bv = norm.rvs(booking_value_mean, booking_value_standard_deviation)
    if verbose:
        print (str(datetime.datetime.now()), bv)
    return bv
    

In [23]:
read_from_file = True

if read_from_file:
    bookings_df=pd.read_csv('SampleData/bookings_df_V'+version+'_'+split_name+'.csv')        
    sessions_df=pd.read_csv('SampleData/sessions_df_V'+version+'_'+split_name+'.csv')   
    
    ## Sample data relates to this Split 
    ## https://app.split.io/org/a91fe850-2994-11e9-a67b-069aee18f4aa/ws/a92650f0-2994-11e9-a67b-069aee18f4aa/splits/ee1ab360-5615-11e9-8b3b-1268881cf0a2/env/a927fea0-2994-11e9-a67b-069aee18f4aa/results/%7B%22version%22%3A1554300425591%2C%22rule%22%3A%22default%20rule%22%2C%22comparison%22%3A%22on%22%2C%22baseline%22%3A%22off%22%2C%22tags%22%3A%5B%5D%2C%22owners%22%3A%5B%5D%2C%22summaryCard%22%3A%22TOTAL%22%7D
    ## (SplitValidate, V2, default rule, Lizzie's org)
else:
    
    n_users = 10000
    bookings_df=pd.DataFrame(columns = ['userID', 'Treatment', 'sessionID', 'bookingID', 'bookingValue'])
    sessions_df=pd.DataFrame(columns = ['userID', 'Treatment', 'sessionID'])


    for user_ID in tqdm(range(n_users)):
        attributes['userID'] = split_name+'V'+version+'_'+str(user_ID)

        try:        
            # assign user to treatment
            treatment = split.get_treatment(attributes['userID'], split_name, attributes)

        except TimeoutException:
           # The SDK failed to initialize in a second. Abort!
           sys.exit()

        # pick the number of sessions user has
        n_sessions = getSessions()

        for session_ID in range(n_sessions):
            # track a new user session
            trackEvent = split.track(attributes['userID'], traffic_type, session_event)
            #if user_ID % 100 == 0 : print (user_ID, treatment, session_ID, trackEvent)        
            # pick how many bookings the user makes in this session
            n_bookings = getBookings()

            session_data = {'userID':attributes['userID'], 
                            'Treatment':treatment,
                            'sessionID': attributes['userID']+'_'+str(session_ID), 
                            'bookings': n_bookings}

            sessions_df = sessions_df.append(session_data, ignore_index=True)        


            for booking_ID in range(n_bookings):

                    # pick the value of the booking
                    booking_value = getBookingValue()
                    trackEvent = split.track(attributes['userID'], traffic_type, booking_event_type , booking_value)

                    booking_data = {'userID':attributes['userID'], 
                                    'Treatment':treatment,
                                    'sessionID': attributes['userID']+'_'+str(session_ID), 
                                    'bookingID': attributes['userID']+'_'+str(session_ID)+'_'+str(booking_ID),
                                    'bookingValue': booking_value}

                    bookings_df = bookings_df.append(booking_data, ignore_index=True)

    print (time.strftime("%d/%m/%Y %H:%M:%S"))
    time.sleep(240)
    print ('destroying @ ',time.strftime("%d/%m/%Y %H:%M:%S"))
    split.destroy()                        

    bookings_df.to_csv('SampleData/bookings_df_V'+version+'_'+split_name+'.csv', index=False)        
    sessions_df.to_csv('SampleData/sessions_df_V'+version+'_'+split_name+'.csv', index=False)        

In [9]:
sessions_df.groupby('Treatment').agg({'userID':'nunique','sessionID':'nunique', 'bookings': 'sum'})

,userID,sessionID,bookings
Treatment,,,
off,5040,7528,3822.0
on,4960,7471,3772.0


In [10]:
#bookings_df_AA=pd.read_csv('bookings_df_V1_SplitValidate_AA.csv')        
#sessions_df_AA=pd.read_csv('sessions_df_V1_SplitValidate_AA.csv')    

## Calculating the metrics manually

### Users that book
#### fraction of unique users with at least one booking 

In [11]:
getProportionMetric(sessions_df, event='bookings')
#returns: 
# Proportion, 
# N_users_completed_action,
# N_users_didntcomplete_action, 
# N_users_total

(0.5119, 5119, 4881, 10000)

### Average Booking value per user

In [12]:
getAverageValue(bookings_df, event = 'bookingID',  value='bookingValue')
# returns: 
# averageValue, 
# averageValue_std,
# averageValue_contributors, 
# averageValue_var, 
# averageValue_min, 
# averageValue_max, 
# averageValue_median, 
# averageValue_95

(100.0755132309858,
 13.315078047475195,
 5119,
 177.29130341035585,
 48.256138697128755,
 149.91091409796317,
 100.02426670814293,
 122.17642325528661)

### Total Booking value per user

In [13]:
getTotalValue(bookings_df,sessions_df, event = 'bookingID',  value='bookingValue')
# returns: 
# averageTotalValue, 
# averageTotalValue_std,
# averageValue_contributors

(76.03915065850505, 94.09828068613025, 10000)

### Bookings per user

In [14]:
getEventsPerUser(bookings_df,sessions_df, event = 'bookingID')
# returns: 
# averageEventsPerUser, 
# averageEventsPerUser_std,
# averageEventsPerUser_contributors

(0.7594, 0.92908447398286, 10000)

### Bookings per session

In [15]:
getEventsPerSession(bookings_df,sessions_df, denom_event = 'sessionID', num_event = 'bookingID')
# returns: 
# averageEventsPerEvent, 
# averageEventsPerEvent_std,
# averageEventsPerEvent_contributors

(0.5086233333333332, 0.6274128558615018, 10000)

### Total Bookings

In [16]:
getAcrossBookings(bookings_df, event = 'bookingID')
# returns
# number of events
# number of contributors

(7594, 5119)

## Evaluating results

In [17]:
printResults(sessions_df, bookings_df)


on

Users :  4960
Users That Book :  (0.5137096774193548, 2548, 2412, 4960)
Average Booking Value :  (99.87505256569894, 13.509461933658256, 2548, 182.50556173696148, 48.75243172134658, 149.91091409796317, 99.61944641356686, 122.50780929050515)
Total Booking Value :  (75.94568347055599, 93.62582712830577, 4960)
Bookings Per User :  0.760483870967742
Bookings Per Session :  0.5038877688172045
Total Bookings in Treatment :  (3772, 2548)
Total Sessions in Treatment :  (7471, 4960)

off

Users :  5040
Users That Book :  (0.5101190476190476, 2571, 2469, 5040)
Average Booking Value :  (100.27418058810392, 13.119198528585756, 2571, 172.11337003244665, 48.256138697128755, 149.32471394676682, 100.31627848171306, 121.6812669105056)
Total Booking Value :  (76.1311342402963, 94.57012957743675, 5040)
Bookings Per User :  0.7583333333333333
Bookings Per Session :  0.51328373015873
Total Bookings in Treatment :  (3822, 2571)
Total Sessions in Treatment :  (7528, 5040)


In [18]:
getStatTestResults(sessions_df, bookings_df, alpha_test = p_thresh, type_two = default_typ2)

**Users That Book**


	b = 51.0119%, c = 51.3710%
	impact = 0.7039%, error margin = 3.2507%, t-error = 3.8416%, MLDE (relative) = 4.873%
	p = 0.7345 (Not Significant)



**Average Booking Value**


	b = 100.2742, c = 99.8751
	impact = -0.3980%, error margin = 0.6123%, relative = 0.6107%, MLDE (relative) = 0.9095%, 
	p = 0.2836 (Not Significant) 
	std (b, c) = (13.1192, 13.5095)



**Total Booking Value**

...


	b = 0.5133, c = 0.5039
	impact = -1.8306%, error margin = 0.0206%, relative = 4.0218%, MLDE (relative) = 6.2026%, 
	p = 0.4540 (Not Significant) 
	std (b, c) = (0.6401, 0.6142)



**Total Bookings in Treatment**


	b =  3822, c =  3772 
	impact = -1.3082% 



**Total Sessions in Treatment**


	b = 7528, c = 7471 
	impact = -0.7572% 



In [22]:
getStatTestResults(sessions_df, bookings_df, alpha_test = 0.1, type_two = 0.1)

**Users That Book**


	b = 51.0119%, c = 51.3710%
	impact = 0.7039%, error margin = 3.2507%, t-error = 3.8416%, MLDE (relative) = 5.736%
	p = 0.7345 (Not Significant)



**Average Booking Value**


	b = 100.2742, c = 99.8751
	impact = -0.3980%, error margin = 0.6123%, relative = 0.6107%, MLDE (relative) = 1.0704%, 
	p = 0.2836 (Not Significant) 
	std (b, c) = (13.1192, 13.5095)



**Total Booking Value**

...


	b = 0.5133, c = 0.5039
	impact = -1.8306%, error margin = 0.0206%, relative = 4.0218%, MLDE (relative) = 7.3000%, 
	p = 0.4540 (Not Significant) 
	std (b, c) = (0.6401, 0.6142)



**Total Bookings in Treatment**


	b =  3822, c =  3772 
	impact = -1.3082% 



**Total Sessions in Treatment**


	b = 7528, c = 7471 
	impact = -0.7572% 

